<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github//jayralencar/pictoBERT/blob/main/Training_PictoBERT.ipynb "><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/jayralencar/pictoBERT/blob/main/Training_PictoBERT.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Training PictoBERT

This notebook contains the code for training pictoBERT. As its aim is to illustrate, we use only one version (contextualized) and train in fewer epochs (2). But, if the objective is to replicate the process described in the paper, it can be changed in the following cell.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Mar 22 17:43:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   45C    P8    15W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch

MODEL_VERSION = "contextual"
MAX_EPOCHS = 10
WARMUP_STEPS = int(MAX_EPOCHS * 0.15)
BATCH_SIZE = 112 # batch size tunned for Tesla T4 GPU
LEARNING_RATE = 1e-04
NUM_WORKERS = 4
GPUS = torch.cuda.device_count()
PRECISION = 16 if torch.cuda.device_count() > 0 else 32
MLM_PROBABILITY= 0.15

In [ ]:
!pip install pytorch_lightning==1.2.10 

     |████████████████████████████████| 849kB 32.0MB/s eta 0:00:01
     |████████████████████████████████| 829kB 69.1MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 32.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 43.4MB/s eta 0:00:01
     |████████████████████████████████| 184kB 73.3MB/s eta 0:00:01
     |████████████████████████████████| 143kB 72.2MB/s eta 0:00:01
     |████████████████████████████████| 296kB 70.1MB/s eta 0:00:01
     |████████████████████████████████| 163kB 67.8MB/s eta 0:00:01
     |████████████████████████████████| 788kB 69.5MB/s eta 0:00:01
     |████████████████████████████████| 133kB 69.0MB/s eta 0:00:01
     |████████████████████████████████| 4.9MB 67.7MB/s eta 0:00:01
     |████████████████████████████████| 4.0MB 63.2MB/s eta 0:00:01
     |████████████████████████████████| 102kB 44.4MB/s ta 0:00:01
     |████████████████████████████████| 163kB 71.6MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 65.4MB/s eta 0:00:

     |████████████████████████████████| 163kB 74.1MB/s eta 0:00:01
     |████████████████████████████████| 276kB 66.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 70.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 31.2MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 153kB 71.7MB/s eta 0:00:01
     |████████████████████████████████| 81kB 42.9MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/e7/67/fd/4e3843a89ae0e233d79452f8f04e45814f350173a99f5addc9
Successfully built multidict
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: tqdm 4.32.1
    Uninstalling tqdm-4.32.1:
      Successfully uninstalled tqdm-4.32.1


## Loading tokenizer

Its necessary to load the trained tokenizer to be used in data collation



In [ ]:
!wget http://jayr.clubedosgeeks.com.br/pictobert/childes_all_new.json

Downloading...
From: https://drive.google.com/uc?id=1-2g-GCxjBwESqDn3JByAJABU9Dkuqy0m
To: /root/capsule/childes_all_new.json
100%|████████████████████████████████████████| 332k/332k [00:00<00:00, 10.3MB/s]


In [ ]:
TOKENIZER_PATH = "./childes_all_new.json" # you can change this path to use your custom tokenizer

from transformers import PreTrainedTokenizerFast

loaded_tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_PATH)
loaded_tokenizer.pad_token = "[PAD]"
loaded_tokenizer.sep_token = "[SEP]"
loaded_tokenizer.mask_token = "[MASK]"
loaded_tokenizer.cls_token = "[CLS]"
loaded_tokenizer.unk_token = "[UNK]"

## Loading datasets

We load the dataset split in the dataset preparation step.

In [ ]:
!wget http://jayr.clubedosgeeks.com.br/pictobert/train_childes_all.pt
!wget http://jayr.clubedosgeeks.com.br/pictobert/test_childes_all.pt
!wget http://jayr.clubedosgeeks.com.br/pictobert/val_childes_all.pt

Downloading...
From: https://drive.google.com/uc?id=1Mn2eIeSCBLk8tpU2NwNQblooFcqSbjmh
To: /root/capsule/train_childes_all.pt
100%|████████████████████████████████████████| 247M/247M [00:03<00:00, 81.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qEX2zVxX8xFbBfY4mLSf0qG4wgkHFggD
To: /root/capsule/test_childes_all.pt
100%|██████████████████████████████████████| 2.52M/2.52M [00:00<00:00, 40.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BwPXeP3lKQk-orKAq1bs2C7hg-B1D6QG
To: /root/capsule/val_childes_all.pt
100%|██████████████████████████████████████| 2.52M/2.52M [00:00<00:00, 45.7MB/s]


In [ ]:
from torch.utils.data import Dataset, Subset
from torch import tensor
from sklearn.model_selection import train_test_split

class MyDataset(Dataset):
  def __init__(self, examples):
    self.input_ids = examples['input_ids']
    self.attention_mask = examples['attention_mask']
    self.special_tokens_mask = examples['special_tokens_mask']
  
  def __len__(self):
    return len(self.input_ids)
  
  def __getitem__(self, idx):
    input_ids = tensor(self.input_ids[idx])
    attention_mask = tensor(self.attention_mask[idx])
    special_tokens_mask = tensor(self.special_tokens_mask[idx])

    return {
      "input_ids":input_ids, 
      "attention_mask":attention_mask, 
      "special_tokens_mask":special_tokens_mask
    }


In [ ]:
import pickle

tds = pickle.load(open('./train_childes_all.pt','rb'))
train_dataset = MyDataset(tds)

vds = pickle.load(open('./val_childes_all.pt','rb'))
val_dataset = MyDataset(vds)

tsds = pickle.load(open('./test_childes_all.pt','rb'))
test_dataset = MyDataset(tsds)


### Data collator

Data collation prepare the dataset for training. For masked language modeling it replace 15% (or the value defined in MLM_PROBABILITY) of each batch tokens with a [MASK] token.

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=loaded_tokenizer, mlm_probability=MLM_PROBABILITY)

### Data loaders

The dataloaders split the data in batchs and apply collation.

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=data_collator,
    drop_last = True,
    shuffle=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=data_collator,
    drop_last = True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    collate_fn=data_collator,
    pin_memory=True,
    drop_last = True
)

## Defining optmizer

BERT uses a Adam Weigth Decay Optimizer. The code below was extracted from [here](https://github.com/jcyk/BERT/blob/88982eb2d8fdfb8984a93df2aa00de07f63af82d/adam.py)

## Defining Model

In [ ]:
from argparse import ArgumentParser
import math
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import get_polynomial_decay_schedule_with_warmup
from transformers import AdamW

from transformers import BertLMHeadModel
from transformers import BertForMaskedLM

class LitBertClassifier(pl.LightningModule):
    def __init__(self, pretrained_model_name='bert-large-uncased'):
        super().__init__()
        self.save_hyperparameters()
        self.batch_size = BATCH_SIZE
        self.lr = LEARNING_RATE
        self.train_dataset = train_dataset
        self.bert = BertForMaskedLM.from_pretrained(pretrained_model_name)
      
    
    def freeze_to(self, layers):
      for param in self.bert.bert.encoder.layer[:layers].parameters():
        param.requires_grad = False


    def forward(self, input_ids, attention_mask, labels=None):
        if labels == None:
            return self.bert(
                input_ids=input_ids,
                attention_mask=attention_mask,
            )    
        return self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels = labels
        )

    def training_step(self, batch, batch_idx):
        outputs = self._shared_step(batch, batch_idx)
        loss = outputs[0]

        self.log("train_loss", loss, on_epoch=True, prog_bar=True,)

        return loss

    def train_dataloader(self):
      return DataLoader(self.train_dataset,batch_size=self.batch_size,num_workers=NUM_WORKERS,pin_memory=True,collate_fn=data_collator)


    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
          result = self._shared_step(batch, batch_idx)
          loss = result[0].detach()

          return {
              "val_loss":loss
          }
    
    def validation_epoch_end(self, outputs):
        val_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        self.log("val_loss", val_loss, on_epoch=True, prog_bar=True,)
    
    
    def test_step(self, batch, batch_idx):
        with torch.no_grad():
          result = self._shared_step(batch, batch_idx)
          loss = result[0].detach()
          perplexity = torch.exp(loss)
          self.log("test_ppl", perplexity, on_epoch=True, prog_bar=True,)
          self.log("test_loss", loss, on_epoch=True, prog_bar=True,)

          return {
              "test_ppl":perplexity,
              "test_loss":loss
          }
    
    def _shared_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        outputs = self.forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        return outputs

    def configure_optimizers(self):
      optimizer = AdamW(self.parameters(), lr=self.lr,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
      scheduler = {
          'scheduler': get_polynomial_decay_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS,num_training_steps=MAX_EPOCHS,lr_end=1e-08),
          # 'scheduler': get_linear_schedule_with_warmup(optimizer,num_warmup_steps=WARMUP_STEPS,num_training_steps=MAX_EPOCHS,last_epoch=-1),
          'name': 'lr'
      }
      return [optimizer],[scheduler]

    def backward(self, loss, optimizer, idx):
        loss.backward()


## Logger

In [ ]:
from pytorch_lightning import loggers as pl_loggers

tb_logger = pl_loggers.TensorBoardLogger("./logs",name=MODEL_VERSION)

## Checkpoint

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints",
    filename='bert-large-{epoch:02d}-{val_loss:.2f}',
    mode='min',
    save_last=True
)

## Trainer

In [ ]:
trainer = pl.Trainer(
    accelerator='ddp',
    max_epochs=MAX_EPOCHS,
    logger=tb_logger,
    gpus=GPUS,
    callbacks=[checkpoint_callback],
    precision=PRECISION,
    auto_scale_batch_size="binsearch"
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


## Load model

We use only one model for exemplifying, but both are downloaded in the following cell.

In [ ]:
!wget http://jayr.clubedosgeeks.com.br/pictobert/pictobert-large-contextual.zip
!wget http://jayr.clubedosgeeks.com.br/pictobert/pictobert-large-gloss.zip

Downloading...
From: https://drive.google.com/uc?id=1AurGqnaxZ6JMl91BM9khScOlxGmzJ23z
To: /root/capsule/pictobert-large-contextual.zip
100%|███████████████████████████████████████| 1.18G/1.18G [00:07<00:00, 148MB/s]


In [ ]:
!unzip ./pictobert-large-contextual.zip -d contextual
!unzip ./pictobert-large-gloss.zip -d contextual

Archive:  ./pictobert-large-contextual.zip
Created by ZIP Extractor v4.31 on 7-May-2021 (https://zipextractor.app)
replace contextual/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
model = LitBertClassifier(MODEL_VERSION)

## Train

In [ ]:
trainer.fit(model, train_dataloader,val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1

  | Name | Type            | Params
-----------------------------------------
0 | bert | BertForMaskedLM | 317 M 
-----------------------------------------
317 M     Trainable params
0         Non-trainable params
317 M     Total params
1,271.248 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

[W reducer.cpp:1289] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint...


1

In [ ]:
#trainer.tune(model)

## Test

In [ ]:
trainer.test(model,test_dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 334.57952880859375, 'test_ppl': inf}
--------------------------------------------------------------------------------


[{'test_ppl': inf, 'test_loss': 334.57952880859375}]